# Building  Text-to-SQL capability to Amazon Athena using Amazon Bedrock

- **Use of amazon.titan-embed-text-v1 for creating embedding**
- **Use of Amazon OpenSearch as a vector database**
- **Use of anthropic.claude-v2:1 as base LLM Model**



## Contents

1. [Objective](#Objective)
1. [Background](#Background-(Problem-Description-and-Approach))
1. [Overall Workflow](#Overall-Workflow)
1. [Conclusion](#Conclusion)


## Objective

This notebook shows is to provide the code snippets within an executable flow from [this AWS Blog post](https://aws.amazon.com/blogs/machine-learning/build-a-robust-text-to-sql-solution-generating-complex-queries-self-correcting-and-querying-diverse-data-sources/).

## Background (Problem Description and Approach)

- **Problem statement**: 

Text-to-SQL solutions aim to generate SQL queries from natural language to enable non-technical users to access and analyze data. However, existing solutions face challenges related to ambiguity in natural language, needing to recreate capabilities for different databases, and collecting comprehensive metadata. The proposed solution in the text aims to address these challenges by incorporating metadata from AWS Glue Data Catalog, evaluating and correcting generated SQL queries using Amazon Athena feedback with multi-pass prompting, and leveraging Athena's support for diverse data sources.

- **Our approach**: 

[`RAG`] The [RAG approach](https://aws.amazon.com/what-is/retrieval-augmented-generation/) offers several advantages. First, it gives up-to-date, precise responses. Rather than relying only on fixed, outdated training data, RAG utilizes current external sources to formulate its answers. In this solution, we used RAG to increase the accuracy of table name from [AWS Glue Data Catalog](https://docs.aws.amazon.com/glue/latest/dg/catalog-and-crawler.html). 

[`Vector Store`] [Amazon OpenSearch](https://aws.amazon.com/opensearch-service/) offers three vector engines to choose from, each catering to different use cases.Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. This code bases used [FAISS for similiarity search](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-opensearch-service-vector-query-filters-faiss/).

[`Amazon Athena`] [Amazon Athena](https://aws.amazon.com/athena/) is a serverless, interactive analytics service built on open-source frameworks, supporting open-table and file formats. Athena provides a simplified, flexible way to analyze petabytes of data where it lives. In this solution, we used Amazon Athena as the SQL engine to 

[`Amazon Bedrock`] [Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon via a single API, along with a broad set of capabilities you need to build generative AI applications with security, privacy, and responsible AI.

We used Bedrock with multi-step / multi-pass component which allows the LLM to correct the generated SQL query for accuracy. Here, the generated SQL is sent for syntax errors. We use Athena error messages to enrich our prompt for the LLM for more accurate and effective corrections in the generated SQL.

- *[RAG on AWS](https://aws.amazon.com/what-is/retrieval-augmented-generation/)*
- *[The langchain OpenSearch documentation](https://python.langchain.com/en/latest/ecosystem/opensearch.html)*
- *[Amazon OpenSearch service documentation](https://docs.aws.amazon.com/opensearch-service/index.html)*
- *[Amazon OpenSearch supports efficient vector](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-opensearch-service-vector-query-filters-faiss/)*



---

## Overall Workflow

**Prerequisite**

The following are prerequisites that needs to be accomplised before executing this notebook.
This Notebook can be executed via a Sagemaker instance or via a VS Code editor
- Create a role having access to bedrock, glue,athena, s3,lakeformation. 
- Assign the role to the Sagemaker instance or to the instance where VS Code editor is running
- Glue Database and tables. Provided spark notebook to create.
- An Amazon OpenSearch cluster for storing embeddings.Here Opensearch credenitals are in notebooks. However Opensearch cluster's access credentials (username and password) can be stored in AWS Secrets Mananger by following steps described [here](https://docs.aws.amazon.com/secretsmanager/latest/userguide/managing-secrets.html).

**The  workflow for this notebook is as follows:**
1. Create an S3 bucket with the name "knowledgebase-<ACCOUNT_ID>" 
    - create a folder "input" in that bucket
2. Download data from source 
    - https://developer.imdb.com/non-commercial-datasets/#titleakastsvgz and upload to S3 bucket from step 1 and into the "input" folder
    - https://developer.imdb.com/non-commercial-datasets/#titlebasicstsvgz and upload to S3 bucket from step 1 and into the "input" folder
3. Glue Steps
    - Create a glue database  "imdb_stg" 
    - Create a glue crawler "text-2-sql-crawler" with the datasource set to the S3 bucket created in step 1.   Run the crawler.
    - 2 tables should be created in Glue  data catalo.g  Make sure you are able to  query through athena. 
4.  From the Bedrock console, Create a new knowledgebase 
1. Install the required Python packages 
1. Create embedding and vector store. Do a similarity search with embeddings stored in the OpenSearch index for an input query.
1. Execute this notebook to generate sql..

## Step 1: Setup
Install the required packages.

In [1]:
!pip3 install boto3
!pip3 install jq

!pip3 install langchain
!pip3 install langchain-community langchain-core
!pip3 install pandas
!pip3 install opensearch-py
!pip3 install langchain-aws
!pip3 install requests-aws4auth
!pip3 install botocore


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached jq-1.7.0.tar.gz (2.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build jq


  error: subprocess-exited-with-error
  
  × Building wheel for jq (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      Executing: ./configure CFLAGS=-fPIC -pthread --disable-maintainer-mode --with-oniguruma=builtin
      error: [WinError 2] The system cannot find the file specified
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for jq
ERROR: Could not build wheels for jq, which is required to install pyproject.toml-based projects

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Step 2: Import all modules. There are some modules in other folder.

In [3]:
import boto3
from botocore.config import Config
from langchain_community.embeddings import BedrockEmbeddings
from langchain_aws import BedrockLLM
import traceback

In [4]:
import logging 
import json
import os,sys
import re
import time
import pandas as pd
import io

In [5]:
from boto_client import Clientmodules
from llm_basemodel import LanguageModel
from athena_execution import AthenaQueryExecute
from openSearchVCEmbedding import EmbeddingBedrockOpenSearch

In [6]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

## Step 3: Checking access to Bedrock

In [5]:
session = boto3.session.Session()
bedrock_client = session.client('bedrock')
print(bedrock_client.list_foundation_models()['modelSummaries'][0])

{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large', 'modelId': 'amazon.titan-tg1-large', 'modelName': 'Titan Text Large', 'providerName': 'Amazon', 'inputModalities': ['TEXT'], 'outputModalities': ['TEXT'], 'responseStreamingSupported': True, 'customizationsSupported': [], 'inferenceTypesSupported': ['ON_DEMAND'], 'modelLifecycle': {'status': 'ACTIVE'}}


## Step 4: Invoking Athena and Bedrock

In [9]:
rqstath=AthenaQueryExecute()

athena client created 
s3 client created !!


### Step 4.1 Update the variables

In [8]:

index_name = 'bedrock-knowledge-base-default-index'  
domain = 'https://wi3kkhxignse60pcjop5.us-east-1.aoss.amazonaws.com'
vector_name = 'bedrock-knowledge-base-default-vector'
fieldname = 'id'
   

In [10]:
ebropen2=EmbeddingBedrockOpenSearch(domain,  vector_name,  fieldname)
if ebropen2 is None:
    print("ebropen2 is null")
else:
    attrs = vars(ebropen2)
    print(', '.join("%s: %s" % item for item in attrs.items()))

bedrock runtime client created 


bedrockllm
bedrock_client: <botocore.client.BedrockRuntime object at 0x000001A2FDAC9580>, language_model: <llm_basemodel.LanguageModel object at 0x000001A2FB3E5AF0>, llm: client=<botocore.client.BedrockRuntime object at 0x000001A2FDAC9580> model_id='anthropic.claude-3-sonnet-20240229-v1:0' model_kwargs={'temperature': 0, 'top_k': 20, 'top_p': 1, 'stop_sequences': ['\n\nHuman:']}, embeddings: client=<botocore.client.BedrockRuntime object at 0x000001A2FDAC9580> region_name=None credentials_profile_name=None model_id='amazon.titan-embed-text-v1' model_kwargs=None endpoint_url=None normalize=False, opensearch_domain_endpoint: https://wi3kkhxignse60pcjop5.us-east-1.aoss.amazonaws.com, http_auth: <requests_aws4auth.aws4auth.AWS4Auth object at 0x000001A2FB283290>, vector_name: bedrock-knowledge-base-default-vector, fieldname: id


## Step 5: Core logic
1. getEmbeddding : Take the input user query and vector search to find the schema from vector db created.
2. generate_sql: Taking the input prompt, generate sql . syntax_checker helps to check the sql syntax.


In [11]:
class RequestQueryBedrock:
    def __init__(self, ebropen2):
    
        ##self.bedrock_client = Clientmodules.createBedrockRuntimeClient()
        self.ebropen2 = ebropen2
  

        self.bedrock_client = ebropen2.bedrock_client
        if self.bedrock_client is None:
            self.bedrock_client = Clientmodules.createBedrockRuntimeClient()
        else : 
            print("the bedrock_client is not null")
        self.language_model = LanguageModel(self.bedrock_client)
        self.llm = self.language_model.llm
        
    def getOpenSearchEmbedding(self, index_name,user_query):
        vcindxdoc=self.ebropen2.getDocumentfromIndex(index_name=index_name)
        documnet=self.ebropen2.getSimilaritySearch(user_query,vcindxdoc)
        #return self.ebropen2.format_metadata(documnet)
        return self.ebropen2.get_data(documnet)
        
    def generate_sql(self,prompt, max_attempt=4) ->str:
            """
            Generate and Validate SQL query.

            Args:
            - prompt (str): Prompt is user input and metadata from Rag to generating SQL.
            - max_attempt (int): Maximum number of attempts correct the syntax SQL.

            Returns:
            - string: Sql query is returned .
            """
            attempt = 0
            error_messages = []
            prompts = [prompt]

            while attempt < max_attempt:
                logger.info(f'Sql Generation attempt Count: {attempt+1}')
                try:
                    logger.info(f'we are in Try block to generate the sql and count is :{attempt+1}')
                    generated_sql = self.llm.predict(prompt)
                    query_str = generated_sql.split("```")[1]
                    query_str = " ".join(query_str.split("\n")).strip()                    
                    sql_query = query_str[3:] if query_str.startswith("sql") else query_str
                    print(sql_query)
                    # return sql_query
                    syntaxcheckmsg=rqstath.syntax_checker(sql_query)
                    if syntaxcheckmsg=='Passed':
                        logger.info(f'syntax checked for query passed in attempt number :{attempt+1}')
                        return sql_query
                    else:
                        prompt = f"""{prompt}
                        This is syntax error: {syntaxcheckmsg}. 
                        To correct this, please generate an alternative SQL query which will correct the syntax error.
                        The updated query should take care of all the syntax issues encountered.
                        Follow the instructions mentioned above to remediate the error. 
                        Update the below SQL query to resolve the issue:
                        {sqlgenerated}
                        Make sure the updated SQL query aligns with the requirements provided in the initial question."""
                        prompts.append(prompt)
                        attempt += 1
                except Exception as e:
                    print(e)
                    logger.error('FAILED')
                    msg = str(e)
                    error_messages.append(msg)
                    attempt += 1
            return sql_query

Create an instance of  RequestQueryBedrock class

In [12]:
rqst=RequestQueryBedrock(ebropen2)

the bedrock_client is not null
bedrockllm


In [13]:
def userinput(user_query):
    logger.info(f'Searching metadata from vector store')
    # vector_search_match=rqst.getEmbeddding(user_query)
    vector_search_match=rqst.getOpenSearchEmbedding( index_name,user_query)
    print("Vector SEARCH MATCH")
    print(vector_search_match)
    details="It is important that the SQL query complies with Athena syntax. During join if column name are same please use alias ex llm.customer_id in select statement. It is also important to respect the type of columns: if a column is string, the value should be enclosed in quotes. If you are writing CTEs then include all the required columns. While concatenating a non string column, make sure cast the column to string. For date columns comparing to string , please cast the string input. Alwayws use the database name along with the table name"
    final_question = "\n\nHuman:"+details + vector_search_match + user_query+ "n\nAssistant:"
    print("FINAL QUESTION :::" + final_question)
    answer = rqst.generate_sql(final_question)
    return answer

## Step 6: User input in Natural Language

In [24]:
#user_query='how many titles exist '
#user_query = 'show me top 10 title by user rating'
#user_query = 'show me top 10 titles in US region'
#user_query = 'which year was a movie/title  made'
user_query = 'how many titles are from the US region'

In [25]:
querygenerated=userinput(user_query)


Searching metadata from vector store
Sql Generation attempt Count: 1
we are in Try block to generate the sql and count is :1


Vector SEARCH MATCH
{"Name": "title","Type": "string","Comment": "The localized title"},{"Name": "region","Type": "string","Comment": "region for this version of the title"},{"Name": "language","Type": "string","Comment": "the language of the title"},{"Name": "types","Type": "string","Comment": "Enumerated set of attributes for this alternative title. One or more of the following: alternative, dvd, festival, tv, video, working, original, imdbDisplay. New values may be added in the future without warning"},{"Name": "attributes","Type": "string","Comment": "Additional terms to describe this alternative title, not enumerated"},{"Name": "isOriginalTitle","Type": "string","Comment": "0: not original title; 1: original title"}]},{"database_name": "text2sql","table_name": "title_rating","table_description": "This dataset contains ratings details"
[{"database_name": "text2sql","table_name": "title","table_description": "This dataset contains title details.","relationships": {"primary_key": [{ 

syntax checked for query passed in attempt number :1


Status : {'State': 'SUCCEEDED', 'SubmissionDateTime': datetime.datetime(2024, 7, 30, 13, 58, 55, 758000, tzinfo=tzlocal()), 'CompletionDateTime': datetime.datetime(2024, 7, 30, 13, 58, 56, 247000, tzinfo=tzlocal())}


## Step 7: Sql Query and Query Execution output

In [26]:
import pprint
my_printer = pprint.PrettyPrinter()
my_printer.pprint(querygenerated)

(' SELECT COUNT(*) AS us_title_count FROM text2sql.title t WHERE t.region = '
 "'US';")


In [27]:
QueryOutput=rqstath.execute_query(querygenerated)

Executing:  SELECT COUNT(*) AS us_title_count FROM text2sql.title t WHERE t.region = 'US';


checking for file :athena_output/ab7b9696-0198-44ca-a625-90567bc0f958.csv


Calling download fine with params ./tmp/athena_output/ab7b9696-0198-44ca-a625-90567bc0f958.csv, {'OutputLocation': 's3://vishal-bucket103/athena_output'}


In [28]:
print(QueryOutput)

   us_title_count
0         1534894



## Cleanup

To avoid incurring future charges, delete the resources.


## Conclusion
In this notebook we were able to see how to use bedrock to deploy LLM Model to generate embeddings,then ingest those embeddings into OpenSearch and finally do a similarity search for user input to the documents (embeddings) stored in OpenSearch. Please read our [AWS Blog Post](https://aws.amazon.com/blogs/machine-learning/build-a-robust-text-to-sql-solution-generating-complex-queries-self-correcting-and-querying-diverse-data-sources/) on this topic to learn more about the solution.
